# Load test data into a remote database

In [1]:
import os

DB_NAME = os.environ["CONTENT_CURATION_POSTGRES_DB_NAME"]
DB_USER = os.environ["CONTENT_CURATION_POSTGRES_USER"]
DB_PASS = os.environ["CONTENT_CURATION_POSTGRES_PASSWORD"]
DB_HOST = os.environ["CONTENT_CURATION_POSTGRES_HOST"]
DB_PORT = os.environ["CONTENT_CURATION_POSTGRES_PORT"]
DATABASE_URL = f'postgres://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

In [2]:
import psycopg2
conn = psycopg2.connect(DATABASE_URL)

In [3]:
import pandas as pd

df = pd.read_csv("test_post_data.csv", delimiter="\t")

In [4]:
cur = conn.cursor()
for idx,content in df.iterrows():
    cur.execute("""
        INSERT INTO social_post_data (internal_id, post_id, title, embed_html, create_utc) 
            VALUES (%s, %s, %s, %s, %s);
    """, (content["internal_id"], content["post_id"], content["text"], content["embed_html"], content["create_utc"]))
conn.commit()

In [5]:
cur = conn.cursor()

cur.execute("""
    SELECT MAX(internal_id)
    FROM social_post_data
""")
max_id = cur.fetchone()
max_id

(99,)

In [5]:
from collections.abc import Iterable

df = pd.read_csv("test_blip_data.csv", delimiter="\t")

for idx,content in df.iterrows():
    cur.execute("""
        INSERT INTO blip_features (internal_id, features)
            VALUES (%s, %s);
    """, (content["internal_id"], content["features"]))

In [6]:
df = pd.read_csv("test_user_data.csv", delimiter="\t")

for idx,content in df.iterrows():
    cur.execute("""
        INSERT INTO user_credentials (user_id, create_utc, email)
            VALUES (%s, %s, %s);
    """, (content["user_id"], content["create_utc"], content["email"]))

In [8]:
df = pd.read_csv("test_curation_data.csv", delimiter="\t")

for idx,content in df.iterrows():
    cur.execute("""
        INSERT INTO curation_modes (primary_user, curation_id, curation_name, curation_key, create_utc)
            VALUES (%s, %s, %s, %s, %s);
    """, (content["primary_user"], content["curation_id"], content["curation_name"], content["curation_key"], content["create_utc"]))

In [9]:
df = pd.read_csv("test_blip_heads.csv", delimiter="\t")

for idx,content in df.iterrows():
    cur.execute("""
        INSERT INTO blip_curation_heads (curation_id, weight1, weight2, bias1, bias2)
            VALUES (%s, %s, %s, %s, %s);
    """, (content["curation_id"], content["weight1"], content["weight2"], content["bias1"], content["bias2"]))

In [10]:
conn.commit()

In [11]:
conn.close()
cur.close()